In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import preprocessor as p
from tqdm import tqdm

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
df = pd.read_csv('clean_train.csv')
df.head()

,id,label,tweet
0,1,0,father dysfunctional selfish drag kid dysfunct...
1,2,0,thanks lyft credit cant use cause dont offer w...
2,3,0,bihday majesty
3,4,0,model love u take u time ur
4,5,0,factsguide society motivation


In [4]:
df = df.dropna()
#normal = df[df['label']==0]
#hate = df[df['label']==1]

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=5000,ngram_range=(1,2), min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
X = tfidfconverter.fit_transform(df['tweet']).toarray()

In [11]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, df['label'], test_size=0.5, random_state=0)

In [26]:
from tensorflow import keras
from keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
Dense(input_dim = 5000, units = 32, activation = 'relu'),
Dense(units = 64, activation = 'relu'),
Dropout(0.5),
Dense(units = 8, activation = 'relu'),
Dense(units = 4, activation = 'relu'),
Dense(units =1, activation = 'sigmoid'),])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 32)                160032    
_________________________________________________________________
dense_31 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 8)                 520       
_________________________________________________________________
dense_33 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 5         
Total params: 162,705
Trainable params: 162,705
Non-trainable params: 0
________________________________________________

In [27]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 20, epochs = 10, validation_data=(X_test, y_test), class_weight ={0:1.0, 1:10.0})

Epoch 1/10
799/799 [==============================] - 2s 2ms/step - loss: 0.9638 - accuracy: 0.8522 - val_loss: 0.3598 - val_accuracy: 0.8254
Epoch 2/10
799/799 [==============================] - 1s 2ms/step - loss: 0.3587 - accuracy: 0.9102 - val_loss: 0.2128 - val_accuracy: 0.9053
Epoch 3/10
799/799 [==============================] - 1s 2ms/step - loss: 0.1877 - accuracy: 0.9510 - val_loss: 0.2099 - val_accuracy: 0.9126
Epoch 4/10
799/799 [==============================] - 1s 2ms/step - loss: 0.1147 - accuracy: 0.9642 - val_loss: 0.2587 - val_accuracy: 0.9167
Epoch 5/10
799/799 [==============================] - 1s 2ms/step - loss: 0.0782 - accuracy: 0.9747 - val_loss: 0.3004 - val_accuracy: 0.9268
Epoch 6/10
799/799 [==============================] - 1s 2ms/step - loss: 0.0491 - accuracy: 0.9871 - val_loss: 0.3439 - val_accuracy: 0.9420
Epoch 7/10
799/799 [==============================] - 1s 2ms/step - loss: 0.0391 - accuracy: 0.9925 - val_loss: 0.3749 - val_accuracy: 0.9411
Epoch 

In [28]:
predictions = model.predict_classes(X_test)

print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[14378   487]
 [  449   652]]
              precision    recall  f1-score   support

           0       0.97      0.97      0.97     14865
           1       0.57      0.59      0.58      1101

    accuracy                           0.94     15966
   macro avg       0.77      0.78      0.78     15966
weighted avg       0.94      0.94      0.94     15966

0.9413754227733935
